<a href="https://colab.research.google.com/github/nekotanku/colab/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision

## ネットワーク構成とForward計算の定義

In [2]:
import numpy as np
! pip install numpy
!pip install scikit-learn

In [44]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

#network構成とForward計算の定義
class CNN(nn.Module):
    def __init__(self, num_classes):
        """
        Convolutional Neural Network
        
        ネットワーク構成：
            input - CONV - CONV - MaxPool - CONV - CONV - MaxPool - FC - output
            ※MaxPoolの直後にバッチ正規化を実施
 
        引数：
            num_classes: 分類するクラス数（＝出力層のユニット数）
        """
        super(CNN, self).__init__()#nn.Moduleを継承する
        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1), #出力サイズ:チャネル=16, 高さ=27, 幅=27 
            nn.BatchNorm2d(16)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=1), #出力サイズ：チャネル=32, 高さ=26, 幅=26
            nn.BatchNorm2d(32)
        )
        self.full_connection = nn.Sequential(
            nn.Linear(in_features=32 * 26 * 26, out_features=512), #in_fueaturesは直前の出力ユニット数
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=512, out_features=num_classes)
        )

#forward計算の定義
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)

        x = x.view(x.size(0), 32 * 26 * 26)

        y = self.full_connection(x)

        return y


①：torch.nn.Moduleを継承してネットワーク構成を定義していく
→お作法

②：forward関数の中で入力xに応じて動的に処理を変更できる
→Define by run

③：ユニット数の計算が面倒

畳み込み層から全結合層に変わる部分は全結合側のユニット数(in_features)を何個に設定するかを計算しないといけない

## 学習用の関数の定義

In [16]:
def train(loader_train, model_obj, optimizer, loss_fn, device, total_epoch, epoch):
    #モデルを学習モードに変更
    model_obj.train()
    
    #ミニバッチごとに学習
    for data, targets in loader_train:
        #GPUを使用するため、to()で明示的に指定
        data = data.to(device)
        targets = targets.to(device)
        #勾配を初期化
        optimizer.zero_grad()
        #順伝播の計算
        outputs = model_obj(data)
        #誤差の計算
        loss = loss_fn(outputs, targets)

        #誤差を逆伝播する
        loss.backward()
        #重みを更新する
        optimizer.step()
    print('Epoch [%d%d], loss: %.4f' % (epoch, total_epoch, loss.item()))

①学習モードの設定を行う

→pytorchは学習時と推論時で各モードを設定する必要がある

②GPUを使用する場合は明示的に指定する

→pytrochではGPUを使用する場合明示的にしていしなければならない


## テスト用の関数の定義

In [17]:
def test(loader_test, trained_model, device):
    #モデルを推論モードに変更
    trained_model.eval()
    correct = 0

    #ミニバッチごとに推論
    with torch.no_grad():
        for data, targets in loader_test:
            #GPUを使用するため、to()で明示的に指定
            data = data.to(device)
            targets = targets.to(device)

            #順伝播の計算
            outputs  = trained_model(data)
            
            #推論結果の取得と正誤判定
            #確率が最大のラベルを取得
            _, predicted = torch.max(outputs.data, 1)
            #正解ならば正解数カウントアップ
            correct += predicted.eq(targets.data.view_as(predicted)).sum()
    
    #正解率を計算
    #テストデータの総数
    data_num = len(loader_test.dataset)
    print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

①推論モードに変更する

②GPUを使用する場合は明示的に指定する

③推論と評価の処理は個別に記述

## main処理の定義

In [42]:
def main():

    #1, GPUの設定
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)

    #2, パラメータの設定
    batch_size = 100
    num_classes = 10
    epochs = 3

    #3, MNISTデータセットを取得
    from sklearn.datasets import fetch_openml
    mnist_X, mnist_y = fetch_openml('mnist_784', version=1, data_home=".", return_X_y=True)

    #データの設定（入力データは閉区間[0,1]に正規化する）
    import numpy as np
    x = mnist_X.astype(np.float32) / 255
    y = mnist_y.astype(np.int32) 

    #5.Dataloaderの作成
    from torch.utils.data import TensorDataset, DataLoader
    from sklearn.model_selection import train_test_split

    #学習用とテスト用に分割
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/7, random_state=0)

    # データのフォーマットを変換：pytorchでの形式=[画像数、チャネル数、高さ、幅]
    x_train = x_train.reshape(60000, 1, 28, 28)
    x_test = x_test.reshape(10000, 1, 28, 28)

    #Pytorchのテンソルに変換
    x_train = torch.Tensor(x_train)
    x_test = torch.Tensor(x_test)
    y_train = torch.LongTensor(y_train)
    y_test = torch.LongTensor(y_test)

    #入力とラベルを組み合わせて最終的なデータを作成
    ds_train = TensorDataset(x_train, y_train)
    ds_test = TensorDataset(x_test, y_test)

    #Dataloaderを作成
    loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
    loader_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)

    # 6. モデル作成
    model = CNN(num_classes=num_classes).to(device)
    print(model) # ネットワークの詳細を確認用に表示

    #7,損失関数を定義
    loss_fn = nn.CrossEntropyLoss()

    #8,最適化手法を定義
    from torch import optim
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    #9,学習
    print('Begin train')
    for epoch in range(1, epochs+1):
        train(loader_train, model, optimizer, loss_fn, device, epochs, epoch)
        test(loader_test, model, device)


①入力のオグジェクトサイズは[画像数、チャネル数、縦、幅]

②DataLoaderを作成する

In [45]:
main()


cpu
CNN(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (full_connection): Sequential(
    (0): Linear(in_features=21632, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_feature